In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By


url = "https://www2.bot.com.tw/house/"
try:
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")

    driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")
    driver.set_page_load_timeout(60)
    driver.get(url)
    
    # 定位日期輸入欄位，並輸入日期
    element = driver.find_element_by_id("fromdate_TextBox") # fromdate_TextBox是網站上"起日"欄位的名字
    element.send_keys("1010101") # 101/01/01
    element = driver.find_element_by_id("todate_TextBox") # fromdate_TextBox是網站上"迄日"欄位的名字
    element.send_keys("1090101") # 109/01/01
    
    # 選定選單索在欄位並點擊
    driver.find_element_by_id("purpose_DDL").click()
    
    # 巡覽選單，點擊對應選項
    for option in driver.find_elements_by_tag_name("option"):
        if option.text == "其他":
            option.click()
            
    #點擊送出按鈕
    element = driver.find_element_by_id("Submit_Button").click()
    
    #等待目標表格出現
    element = WebDriverWait(driver, 5).until(
        expected_conditions.presence_of_element_located((By.ID, "House_GridView"))
    )
    
    # page_source 可以回傳目前瀏覽器所看到的網頁文件
    soup = BeautifulSoup(driver.page_source, "html5lib")
    table = soup.find(id = "House_GridView")
    for row in table.find_all("tr"):
        print([s for s in row.stripped_strings])

    next_page = soup.find(id = "lblNextoneBottom")
    print(next_page.attrs)
    while True:
        if "disabled" in next_page.attrs: # attrs 為 "find"功能 之回應物件的屬性
            break
        else:
            driver.find_element_by_id("lblNextoneBottom").click()
            soup = BeautifulSoup(driver.page_source, "html5lib")
            table = soup.find(id = "House_GridView")
            for row in table.find_all("tr"):
                print([s for s in row.stripped_strings])
            next_page = soup.find(id = "lblNextoneBottom")
finally:
    driver.quit() # 關閉瀏覽器, 結束 webdriver process            

['管理編號', '拍賣日期', '門\u3000牌', '用途', '樓層別', '建坪', '地坪', '拍賣總價', '執行法院', '案\u3000號', '拍賣次數', '備\u3000註']
['J014080005', '108.12.31', '其他', '48.40', '257,000', '嘉義地院', '107司執字第44984號', '4', '特別推薦']
['J010060003', '108.11.26', '其他', '47.67', '524,400', '彰化地院', '104年度司執字第31985號', '減', '特別推薦']
['J241080007', '108.11.06', '其他', '1,516,200', '臺灣士林地方法院民事執行處(委託金服)', '107年度司執春字第68517號', '4', '特別推薦']
['J241080006', '108.09.05', '其他', '0', '臺灣士林地方法院民事執行處', '107年度司執春字第68517號', '2', '特別推薦']
['J092080002', '108.03.28', '台中市西屯區中工三路89之2號', '其他', '2樓', '52.50', '9.68', '5,760,000', '台中地院', '中院麟民執106司執辰字第110012號', '3', '特別推薦']
['J165080001', '108.01.30', '台北市中正區重慶南路3段67之1號及增建部份', '其他', '1', '21.68', '22.08', '81,830,000', '台灣金服', '107北金職一字第11號', '1', '可點交']
['J092070006', '107.12.27', '臺中市西屯區中工三路89之2號', '其他', '52.50', '9.68', '8,000,000', '台中地院', '台中地院106年度司執字第110012號', '1']
['J165070001', '107.11.16', '新北市新莊區中正路375之1號', '其他', '1', '41.64', '5.90', '9,486,000', '台灣金服', '107板金職四字第150號', '公告應買', '特別推薦', '可點交